In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import TD3_tf2
import preprocess
np.set_printoptions(precision=6)

### Effortless implementation of the pre-trained model
#### **(User-input required)**<br>  

This allows users to use the pre-trained model to trade EUR/USD daily.  
Simply follow the steps below:  

1. Acquire the daily historical EUR/USD price data of the past 55 trading days or more, including the present day, in a .csv format. The columns should represent ```['Bid Open', 'Bid High', 'Bid Low', 'Bid Close', 'Ask Open', 'Ask High', 'Ask Low', 'Ask Close']```<br><br>  

    e.g. To estimate the optimal action for 2021-10-21, the price data can be:<br><br>  

    | Index | Bid Open | Bid High | Bid Low | Bid Close | Ask Open | Ask High | Ask Low | Ask Close |
    | :---: | :------: | :------: | :-----: | :-------: | :------: | :------: | :-----: | :-------: |
    | 2021-08-19 | ... | ... | ... | ... | ... | ... | ... | ... |
    | 2021-08-20 | ... | ... | ... | ... | ... | ... | ... | ... |
    |     ...    | ... | ... | ... | ... | ... | ... | ... | ... |
    | 2021-10-21 | ... | ... | ... | ... | ... | ... | ... | ... |  

<br>  

2. Provide the path to the .csv file by setting ```dataset_path```<br><br>  

    e.g. ```dataset_path = 'EURUSD_last_55days.csv'```

<br>  

3. Run all cells, the return value is the current estimated optimal action to be taken, generated by the pre-trained model.

In [ ]:
# user-input
dataset_path = 

In [ ]:
def one_step_implementation(df):
    df = df.copy()
    if df.shape[0] < 55:
        print("The length of the dataset is too short.")
        return
    agent_obs_data = df.copy()
    agent_obs_data.columns = ['Open', 'High', 'Low', 'Close', 'AskOpen', 'AskHigh', 'AskLow', 'AskClose']
    agent_obs_data = preprocess.ma(df=agent_obs_data, ma_ranges=[10, 21, 50])
    agent_obs_data = preprocess.macd(df=agent_obs_data, short_long_signals=[(12, 26, 9)])
    agent_obs_data = preprocess.full_stochastic(df=agent_obs_data, stochastic_ranges=[(14, 3, 3)])
    agent_obs_data = preprocess.rsi(df=agent_obs_data, rsi_ranges=[14])
    agent_obs_data = preprocess.bollinger_bands(df=agent_obs_data,
                                                bollinger_period_sd_ranges=[(20, 2)])
    agent_obs_data = agent_obs_data.iloc[:, 8:]
    agent_obs_data = preprocess.add_lag(df=agent_obs_data, lag=5)
    agent_obs_data = agent_obs_data.dropna()

    mins = np.load("samples/sample_mins.npy")
    maxs = np.load("samples/sample_maxs.npy")

    agent_obs_data = agent_obs_data.clip(mins, maxs, axis=1)

    agent_obs_data = (agent_obs_data - mins) / (maxs - mins)

    current_balance = np.reshape(1, (1, -1))
    previous_actions = np.reshape(0, (1, -1))
    market_observation = np.array(agent_obs_data)[-1].reshape((1, -1))
    observation = np.concatenate([current_balance, previous_actions, market_observation], axis=-1)
    observation = tf.cast(observation, dtype=tf.float32)

    final_actor = TD3_tf2.ActorNetwork(fc_dim=1024, num_layers=4, n_actions=1)
    _ = final_actor(observation)
    final_actor.load_weights("samples/sample_actor.h5")

    action = final_actor(observation)
    long = bool(action[0].numpy()>0)
    short = bool(action[0].numpy()<0)
    p_action = np.absolute(action[0,0].numpy()*0.001)
    print(f"Estimated optimal action now: EUR/USD {'Long'*long}{'Short'*short} {p_action:.4}% of balance")

    return action[0]

In [ ]:
history = pd.read_csv(str(dataset_path), index_col=0)
actions = one_step_implementation(history)

<br>  

*The Pre-trained model was trained on price data provided by a broker that may or may not be different to which of the user's, the performance and the estimated action may and is likely to vary.*<br><br>  